# LOL player cost model

In [ ]:
# Load the data
import os
import pandas as pd

LOL_COST_FILE_PATH = os.path.join(
    os.environ["FANTASY_ARCHIVE_BASE"], "lol", "cost_archive"
)

dk_dfs: list[pd.DataFrame] = []
fd_dfs: list[pd.DataFrame] = []
for filename in os.listdir(LOL_COST_FILE_PATH):
    if "draftkings" in filename:
        df = pd.read_csv(os.path.join(LOL_COST_FILE_PATH, filename))
        df["service"] = "draftkings"
        dk_dfs.append(df)
    elif "fanduel" in filename:
        df = pd.read_csv(os.path.join(LOL_COST_FILE_PATH, filename))
        df["service"] = "fanduel"
        df["filename"] = filename
        fd_dfs.append(df)

dk_df = pd.concat(dk_dfs)
fd_df = pd.concat(fd_dfs)

display(dk_df)
display(fd_df)


In [ ]:
# transform draftkings data
dk_cleaned_df = dk_df[dk_df["Roster Position"] != "CPT"].rename(
    {
        "Position": "pos",
        "Name": "name",
        "Salary": "cost",
        "TeamAbbrev": "team",
    },
    axis=1,
)


def opp_func(x):
    teams = x["Game Info"].split(" ", 1)[0].split("@")
    return teams[1] if teams[0] == x["team"] else teams[0]


dk_cleaned_df["opp"] = dk_cleaned_df.apply(opp_func, axis=1)
dk_cleaned_df["date"] = dk_cleaned_df["Game Info"].apply(lambda x: x.split(" ", 2)[1])
dk_cleaned_df = dk_cleaned_df[["pos", "name", "cost", "team", "service", "opp", "date"]]

display(dk_cleaned_df)


In [ ]:
# transform fanduel data
fd_cleaned_df = fd_df.rename(
    {
        "Position": "pos",
        "First Name": "name",
        "Salary": "cost",
        "Team": "team",
        "Opponent": "opp",
    },
    axis=1,
)
fd_cleaned_df["date"] = fd_cleaned_df["filename"].apply(
    lambda x: x.split("_", 2)[2].split("-", 1)[0]
)
fd_cleaned_df = fd_cleaned_df[
    ["pos", "name", "cost", "team", "service", "opp", "date"]
]
display(fd_cleaned_df)

In [ ]:
# combine data and add points
def points_func(x):
    # find the game based on team, opp and date
    game_id: int
    # get the stat ID based on service and team/player
    stat_id: int
    # storage for player/teams points
    pts_df: pd.DataFrame
    if x.pos == "TEAM":
        # get team id
        # pull all points data for the team (and cache it)
        raise NotImplementedError()
    else:
        # get player id
        # pull all points data for the player (and cache it)
        raise NotImplementedError()
    # calculate points based on date
    pts_1: float
    pts_5: float
    pts_std: float
    return {"pts-1": pts_1, "pts-mean-5": pts_5, "pts-mean-std": pts_std}


data_df = pd.concat([dk_cleaned_df, fd_cleaned_df])
pts_df = data_df.apply(
    points_func, axis=1, result_type="expand"
)
data_df = pd.concat([data_df, pts_df], axis=1)

display(data_df)

In [ ]:
# train/evaluate/export model using automl